In [1]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 110851 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.1-0ubuntu3~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.1-0ubuntu3~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.1-0ubuntu3~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=ht

In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [3]:
cd drive/Houston/

/content/drive/Houston


In [0]:
import numpy as np
import cv2

In [0]:
area_hsi = []
for i in range(3):
    for j in range(14):
        img_path = "area/pc_area_" + str(i) + "_" + str(j) + ".png"
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        area_hsi.append(img)
        
area_hsi = np.array(area_hsi)

In [0]:
area_lidar = []
for i in range(14):
    img_path = "area/lidar_area_" + str(i) + ".png"
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    area_lidar.append(img)

area_lidar = np.array(area_lidar)

In [7]:
print(area_hsi.shape)
print(area_lidar.shape)

(42, 349, 1905)
(14, 349, 1905)


In [0]:
train_file_name = "labels/train.txt"
test_file_name = "labels/test.txt"

In [0]:
file = open(train_file_name)
triplets = file.read().split()
for i in range(0, len(triplets)):
    triplets[i] = triplets[i].split(",")
train_array = np.array(triplets, dtype=int)
file.close()


file = open(test_file_name)
triplets = file.read().split()
for i in range(0, len(triplets)):
    triplets[i] = triplets[i].split(",")
test_array = np.array(triplets, dtype=int)
file.close()

HEIGHT = train_array.shape[0]
WIDTH = train_array.shape[1]

In [0]:
area_hsi_train_data = []
area_hsi_test_data = []
area_lidar_train_data = []
area_lidar_test_data = []
moment_hsi_train_data = []
moment_hsi_test_data = []
moment_lidar_train_data = []
moment_lidar_test_data = []
train_labels = []
test_labels = []
for i in range(HEIGHT):
    for j in range(WIDTH):
        if train_array[i, j] != 0:
            area_hsi_train_data.append(area_hsi[:, i, j])
            area_lidar_train_data.append(area_lidar[:, i, j])
            train_labels.append(train_array[i, j])
        if test_array[i,j] != 0:
            area_hsi_test_data.append(area_hsi[:, i, j])
            area_lidar_test_data.append(area_lidar[:, i, j])
            test_labels.append(test_array[i, j])

In [0]:
area_hsi_train_data = np.array(area_hsi_train_data)
area_hsi_test_data = np.array(area_hsi_test_data)
area_lidar_train_data = np.array(area_lidar_train_data)
area_lidar_test_data = np.array(area_lidar_test_data)
train_labels = np.array(train_labels)
test_labels = np.array(test_labels)

In [12]:
print(area_hsi_train_data.shape)
print(area_hsi_test_data.shape)
print(area_lidar_train_data.shape)
print(area_lidar_test_data.shape)
print(train_labels.shape)
print(test_labels.shape)

(2832, 42)
(12197, 42)
(2832, 14)
(12197, 14)
(2832,)
(12197,)


In [13]:
import keras

train_one_hot = keras.utils.to_categorical(train_labels-1)
test_one_hot = keras.utils.to_categorical(test_labels-1)

Using TensorFlow backend.


In [14]:
print(train_one_hot.shape)
print(test_one_hot.shape)
print(train_one_hot.shape)
print(test_one_hot.shape)

(2832, 15)
(12197, 15)
(2832, 15)
(12197, 15)


In [0]:
HSI_PATCH_SIZE = 27
LiDAR_PATCH_SIZE = 41
CONV1 = 500
CONV2 = 100
FC1 = 200
FC2 = 84
LEARNING_RATE = 0.005
BATCH_SIZE = 25

In [0]:
import numpy as np
padded_area_hsi = np.lib.pad(area_hsi, ((0,0), (HSI_PATCH_SIZE//2, HSI_PATCH_SIZE//2), (HSI_PATCH_SIZE//2,HSI_PATCH_SIZE//2)), 'reflect')
padded_area_lidar = np.lib.pad(area_lidar, ((0,0), (LiDAR_PATCH_SIZE//2, LiDAR_PATCH_SIZE//2), (LiDAR_PATCH_SIZE//2,LiDAR_PATCH_SIZE//2)), 'reflect')

In [17]:
print(padded_area_hsi.shape)
print(padded_area_lidar.shape)

(42, 375, 1931)
(14, 389, 1945)


In [0]:
def get_patches(data, patch_size, row, column):
  offset = patch_size // 2
  row_low = row - offset
  row_high = row + offset
  col_low = column - offset
  col_high = column + offset
  return data[0:, row_low:row_high + 1, col_low:col_high + 1].reshape(patch_size, patch_size, data.shape[0])

In [0]:
area_hsi_train_patches = []
area_hsi_test_patches = []
area_lidar_train_patches = []
area_lidar_test_patches = []

for i in range(HEIGHT):
  for j in range(WIDTH):
    if train_array[i, j] != 0:
      area_hsi_train_patches.append(get_patches(padded_area_hsi, HSI_PATCH_SIZE, i+HSI_PATCH_SIZE//2, j+HSI_PATCH_SIZE//2))
      area_lidar_train_patches.append(get_patches(padded_area_lidar, LiDAR_PATCH_SIZE, i+LiDAR_PATCH_SIZE//2, j+LiDAR_PATCH_SIZE//2))
    if test_array[i, j] != 0:
      area_hsi_test_patches.append(get_patches(padded_area_hsi, HSI_PATCH_SIZE, i+HSI_PATCH_SIZE//2, j+HSI_PATCH_SIZE//2))
      area_lidar_test_patches.append(get_patches(padded_area_lidar, LiDAR_PATCH_SIZE, i+LiDAR_PATCH_SIZE//2, j+LiDAR_PATCH_SIZE//2))

In [0]:
area_hsi_train_patches = np.array(area_hsi_train_patches)
area_hsi_test_patches = np.array(area_hsi_test_patches)
area_lidar_train_patches = np.array(area_lidar_train_patches)
area_lidar_test_patches = np.array(area_lidar_test_patches)

In [21]:
print(area_hsi_train_patches.shape)
print(area_hsi_test_patches.shape)
print(area_lidar_train_patches.shape)
print(area_lidar_test_patches.shape)

(2832, 27, 27, 42)
(12197, 27, 27, 42)
(2832, 41, 41, 14)
(12197, 41, 41, 14)


In [0]:
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Conv2D, Dense, Flatten
from tensorflow.python.keras.layers import InputLayer
from tensorflow.python.keras.layers import  MaxPooling2D
from tensorflow.python.keras.layers import BatchNormalization, Dropout
from tensorflow.python.keras.optimizers import Adam,SGD

BANDS = area_hsi_train_patches.shape[3]
NUM_CLS = train_one_hot.shape[1]
BATCH_SIZE = 25

area_hsi_model = Sequential()
area_hsi_model.add(InputLayer(input_shape=(HSI_PATCH_SIZE, HSI_PATCH_SIZE, BANDS)))
area_hsi_model.add(Conv2D(kernel_size=6, strides=2, filters=CONV1, padding='same', activation='relu', name='conv1'))
area_hsi_model.add(BatchNormalization())
area_hsi_model.add(MaxPooling2D(pool_size=2, strides=2))
area_hsi_model.add(Conv2D(kernel_size=5, strides=2, filters=CONV2, padding='same', activation='relu', name='conv2'))
area_hsi_model.add(BatchNormalization())
area_hsi_model.add(MaxPooling2D(pool_size=2, strides=2))


area_hsi_model.add(Flatten())
area_hsi_model.add(Dense(FC1, activation='relu'))
area_hsi_model.add(Dropout(0.6))
area_hsi_model.add(Dense(FC2, activation='relu'))
area_hsi_model.add(Dropout(0.4))
area_hsi_model.add(Dense(NUM_CLS, activation='softmax'))

In [0]:
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Conv2D, Dense, Flatten
from tensorflow.python.keras.layers import InputLayer
from tensorflow.python.keras.layers import  MaxPooling2D
from tensorflow.python.keras.layers import BatchNormalization, Dropout
from tensorflow.python.keras.optimizers import Adam,SGD

BANDS = area_lidar_train_patches.shape[3]
NUM_CLS = train_one_hot.shape[1]


area_lidar_model = Sequential()
area_lidar_model.add(InputLayer(input_shape=(LiDAR_PATCH_SIZE, LiDAR_PATCH_SIZE, BANDS)))
area_lidar_model.add(Conv2D(kernel_size=6, strides=2, filters=CONV1, padding='same', activation='relu', name='conv1'))
area_lidar_model.add(BatchNormalization())
area_lidar_model.add(MaxPooling2D(pool_size=2, strides=2))
area_lidar_model.add(Conv2D(kernel_size=5, strides=2, filters=CONV2, padding='same', activation='relu', name='conv2'))
area_lidar_model.add(BatchNormalization())
area_lidar_model.add(MaxPooling2D(pool_size=2, strides=2))


area_lidar_model.add(Flatten())
area_lidar_model.add(Dense(FC1, activation='relu'))
area_lidar_model.add(Dropout(0.7))
area_lidar_model.add(Dense(FC2, activation='relu'))
area_lidar_model.add(Dropout(0.5))
area_lidar_model.add(Dense(NUM_CLS, activation='softmax'))

In [0]:
area_hsi_model.load_weights('Models/area_hsi_model_weights.h5')
area_lidar_model.load_weights('Models/area_lidar_model_weights.h5')

In [0]:
area_hsi_model.load_weights('Models/area_hsi_model_weights.h5')
area_lidar_model.load_weights('Models/area_lidar_model_weights.h5')

In [0]:
from operator import truediv

def AA_andEachClassAccuracy(confusion_matrix):
    counter = confusion_matrix.shape[0]
    list_diag = np.diag(confusion_matrix)
    list_raw_sum = np.sum(confusion_matrix, axis=1)
    each_acc = np.nan_to_num(truediv(list_diag, list_raw_sum))
    average_acc = np.mean(each_acc)
    return each_acc, average_acc

In [27]:
test_cls = test_labels - 1
prediction = area_hsi_model.predict(area_hsi_test_patches).argmax(axis=-1)



from sklearn import metrics, preprocessing


overall_acc = metrics.accuracy_score(prediction, test_cls)
kappa = metrics.cohen_kappa_score(prediction, test_cls)
confusion_matrix = metrics.confusion_matrix(prediction, test_cls)
each_acc, average_acc = AA_andEachClassAccuracy(confusion_matrix) 


print("Overall Accuracy of testing sapmles : ",overall_acc)
print("Average Accuracy of testing samples : ",average_acc)
print("Kappa statistics of testing samples : ",kappa)
print("Each class accuracy of testing samples : ", each_acc)
print("Confusion matrix :", confusion_matrix)

Overall Accuracy of testing sapmles :  0.6269574485529229
Average Accuracy of testing samples :  0.6591057825766897
Kappa statistics of testing samples :  0.5951054853872203
Each class accuracy of testing samples :  [0.68176915 0.6295045  0.65667575 0.63526245 0.8633218  0.45791246
 0.65377856 0.60416667 0.75607287 0.90238095 0.53174061 0.40479616
 0.54054054 0.80866426 0.76      ]
Confusion matrix : [[632 112   0 130   0   1   0   0   0   0   0   0   0   0  52]
 [133 559   0 158   0   0  29   0   1   0   0   0   0   0   8]
 [  0   0 482   0   0   0   0   0   0   0   0   0   0   0 252]
 [ 57  84   0 472   0   0 123   7   0   0   0   0   0   0   0]
 [  2  13   0  18 998   0   0   3  33   0   0   0   0  16  73]
 [ 47  32   0   4   0 136  24  37   7   0   0   0   0   0  10]
 [  1  86   0 132   4   6 744  46  89   0  11   0  15   4   0]
 [  2   9   0  13   6   0   0 435 110  55   7  52  28   1   2]
 [  0   4   0  16  24   0  51   0 747  56  34  10  46   0   0]
 [  1   2   0  18   1   0   0

In [28]:
test_cls = test_labels - 1
prediction = area_lidar_model.predict(area_lidar_test_patches).argmax(axis=-1)



from sklearn import metrics, preprocessing


overall_acc = metrics.accuracy_score(prediction, test_cls)
kappa = metrics.cohen_kappa_score(prediction, test_cls)
confusion_matrix = metrics.confusion_matrix(prediction, test_cls)
each_acc, average_acc = AA_andEachClassAccuracy(confusion_matrix) 


print("Overall Accuracy of testing sapmles : ",overall_acc)
print("Average Accuracy of testing samples : ",average_acc)
print("Kappa statistics of testing samples : ",kappa)
print("Each class accuracy of testing samples : ", each_acc)
print("Confusion matrix :", confusion_matrix)

Overall Accuracy of testing sapmles :  0.5304583094203492
Average Accuracy of testing samples :  0.5364854006142565
Kappa statistics of testing samples :  0.4906388260322003
Each class accuracy of testing samples :  [0.47682119 0.40447154 0.59138187 0.51807229 0.77912254 0.40659341
 0.51507208 0.56845476 0.33020231 0.64369501 0.735269   0.47503201
 0.36090226 0.98969072 0.2525    ]
Confusion matrix : [[360  60   0 113  33   6  68   4  92   0   8   3   3   5   0]
 [ 81 398   7 123  55   0  86  26 102   0  21  44  19   7  15]
 [  0   1 398   0  41   0  12   0   0   0   0   1   0   0 220]
 [ 10  23   0 258  23   3  39  26  32   2   5  56  17   4   0]
 [ 21  14   0  38 515   3   0   0  45   0  22   0   1   2   0]
 [ 34   0   0  46   3 111   0   0  46  12   0  15   0   6   0]
 [  2  16   5 129  11   4 393  33  92   8   0   2   2   0  66]
 [ 20   0   0 144 110   8  89 710  13  32  22  95   0   6   0]
 [122 199   0 103  75   2 140  19 457  81  73  18  33   2  60]
 [ 20 199   0   5   2   0 127

In [0]:
from tensorflow.python.keras.models import Model
intermediate_layer_area_hsi_model = Model(area_hsi_model.layers[0].input,outputs=area_hsi_model.layers[6].output)
intermediate_layer_area_lidar_model = Model(area_lidar_model.layers[0].input,outputs=area_lidar_model.layers[6].output)

area_hsi_train_flatten = intermediate_layer_area_hsi_model.predict(area_hsi_train_patches)
area_lidar_train_flatten = intermediate_layer_area_lidar_model.predict(area_lidar_train_patches)

area_hsi_test_flatten = intermediate_layer_area_hsi_model.predict(area_hsi_test_patches)
area_lidar_test_flatten = intermediate_layer_area_lidar_model.predict(area_lidar_test_patches)

In [30]:
print(area_hsi_train_flatten.shape)
print(area_lidar_train_flatten.shape)
print(area_hsi_test_flatten.shape)
print(area_lidar_test_flatten.shape)

(2832, 400)
(2832, 400)
(12197, 400)
(12197, 400)


In [0]:
def get_outer_products(v1,v2):
  products = []
  for i in range(v1.shape[0]):
    products.append(np.outer(v1[i], v2[i]))
  return np.array(products)

In [0]:
train_outer_products = get_outer_products(area_hsi_train_flatten, area_lidar_train_flatten)
#test_outer_products = get_outer_products(hsi_test_flatten, lidar_test_flatten)

In [34]:
train_outer_products = train_outer_products.reshape(train_outer_products.shape[0], train_outer_products.shape[1], train_outer_products.shape[2], 1)
print(train_outer_products.shape)
#print(test_outer_products.shape)

(2832, 400, 400, 1)


In [63]:
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Conv2D, Dense, Flatten
from tensorflow.python.keras.layers import InputLayer
from tensorflow.python.keras.layers import  MaxPooling2D
from tensorflow.python.keras.layers import BatchNormalization, Dropout
from tensorflow.python.keras.optimizers import Adam,SGD

BANDS = train_outer_products.shape[3]
NUM_CLS = train_one_hot.shape[1]
BATCH_SIZE = 25

fusion_model = Sequential()
fusion_model.add(InputLayer(input_shape=(train_outer_products.shape[1], train_outer_products.shape[2], BANDS)))
fusion_model.add(Conv2D(kernel_size=12, strides=8, filters=256, padding='same', activation='relu', name='conv1'))
fusion_model.add(BatchNormalization())
fusion_model.add(MaxPooling2D(pool_size=2, strides=3))
fusion_model.add(Conv2D(kernel_size=12, strides=8, filters=128, padding='same', activation='relu', name='conv2'))
fusion_model.add(BatchNormalization())
fusion_model.add(MaxPooling2D(pool_size=2, strides=2))


fusion_model.add(Flatten())
fusion_model.add(Dense(FC1, activation='relu'))
fusion_model.add(Dropout(0.75))
fusion_model.add(Dense(FC2, activation='relu'))
fusion_model.add(Dropout(0.5))
fusion_model.add(Dense(NUM_CLS, activation='softmax'))


fusion_model.summary()

sgd = SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)
fusion_model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1 (Conv2D)               (None, 50, 50, 256)       37120     
_________________________________________________________________
batch_normalization_26 (Batc (None, 50, 50, 256)       1024      
_________________________________________________________________
max_pooling2d_26 (MaxPooling (None, 17, 17, 256)       0         
_________________________________________________________________
conv2 (Conv2D)               (None, 3, 3, 128)         4718720   
_________________________________________________________________
batch_normalization_27 (Batc (None, 3, 3, 128)         512       
_________________________________________________________________
max_pooling2d_27 (MaxPooling (None, 1, 1, 128)         0         
_________________________________________________________________
flatten_13 (Flatten)         (None, 128)               0         
__________

In [62]:
history = fusion_model.fit(train_outer_products, train_one_hot, batch_size=BATCH_SIZE, shuffle=True, epochs=50)

Epoch 1/50
2832/2832 [==============================] - 7s 3ms/step - loss: 0.0232 - acc: 0.9915
Epoch 2/50
2832/2832 [==============================] - 7s 3ms/step - loss: 0.0232 - acc: 0.9933
Epoch 3/50
2832/2832 [==============================] - 9s 3ms/step - loss: 0.0164 - acc: 0.9944
Epoch 4/50
2832/2832 [==============================] - 10s 4ms/step - loss: 0.0216 - acc: 0.9929
Epoch 5/50
2832/2832 [==============================] - 10s 3ms/step - loss: 0.0150 - acc: 0.9954
Epoch 6/50
2832/2832 [==============================] - 7s 2ms/step - loss: 0.0172 - acc: 0.9933
Epoch 7/50
2832/2832 [==============================] - 7s 3ms/step - loss: 0.0293 - acc: 0.9901
Epoch 8/50
2832/2832 [==============================] - 7s 3ms/step - loss: 0.0210 - acc: 0.9933
Epoch 9/50
2832/2832 [==============================] - 7s 3ms/step - loss: 0.0180 - acc: 0.9951
Epoch 10/50
2832/2832 [==============================] - 7s 3ms/step - loss: 0.0171 - acc: 0.9936
Epoch 11/50
2832/2832 [====

In [61]:
test_cls = test_labels - 1

prediction = []



for i in range(12197):
  prediction.append(fusion_model.predict(np.outer(area_hsi_test_flatten[i], area_lidar_test_flatten[i]).reshape(1, 400, 400, 1)).argmax(axis=-1))


prediction = np.array(prediction)


from sklearn import metrics, preprocessing

overall_acc = metrics.accuracy_score(prediction, test_cls)
kappa = metrics.cohen_kappa_score(prediction, test_cls)
confusion_matrix = metrics.confusion_matrix(prediction, test_cls)
each_acc, average_acc = AA_andEachClassAccuracy(confusion_matrix) 


print("Overall Accuracy of training sapmles : ",overall_acc)
print("Average Accuracy of training samples : ",average_acc)
print("Kappa statistics of training samples : ",kappa)
print("Each class accuracy of training samples : ", each_acc)
print("Confusion matrix :\n", confusion_matrix)

Overall Accuracy of training sapmles :  0.6410592768713618
Average Accuracy of training samples :  0.6652570667133558
Kappa statistics of training samples :  0.6106019479985674
Each class accuracy of training samples :  [0.74054983 0.45415908 0.64246575 0.64685908 0.89697543 0.55918367
 0.70856507 0.65943775 0.64808743 0.96685083 0.5323654  0.61353033
 0.3752809  0.88       0.65454545]
Confusion matrix :
 [[431  78   0  23   0   0  11   0   0   0   0   0   0  10  29]
 [381 748   0 326   1   0  77   0  38   0   9  17   0   0  50]
 [  0   0 469   0   0   0   0   0   0   0   0   0   0   0 261]
 [ 28  44   0 381   0   0  71   0   4   0   0   1   0   0  60]
 [  0   0   0  35 949   0   0   3  60   0   3   1   0   7   0]
 [ 33   0   0  31   0 137   2   7   2  33   0   0   0   0   0]
 [  2  31   0  90   0   6 637   6 121   0   0   0   6   0   0]
 [  0   1   0  68  37   0  42 821  34  76   3 152   8   3   0]
 [ 33   8   0  16  10   0 115  17 593  48  45   5  24   0   1]
 [  0   0   0   0   0   